In [32]:
import pandas as pd

data = pd.read_csv("../csv/melb_data.csv")
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [33]:
y = data["Price"]
X = data.drop(["Price"], axis=1)


In [34]:
from sklearn.model_selection import train_test_split

X_train,X_valid,y_train,y_valid = train_test_split(X,y,train_size=0.8,test_size=0.2, random_state=0)

In [35]:
missing_vals_col = [col for col in X_train.columns
                    if X_train[col].isnull().any()]

missing_vals_col

['Car', 'BuildingArea', 'YearBuilt', 'CouncilArea']

In [36]:
X_train.drop(missing_vals_col, axis=1, inplace=True)
X_valid.drop(missing_vals_col,axis=1,inplace=True)

In [57]:
X_train.shape

(10864, 16)

In [55]:
low_card_cols = [col for col in X_train.columns
                 if X_train[col].nunique() < 10 and X_train[col].dtype == "object"]

numerical_cols = [col for col in X_train.columns
                  if X_train[col].dtype in ["int64", "float64"]]



In [53]:
low_card_cols

['Type', 'Method', 'Regionname']

In [56]:
numerical_cols

['Rooms',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Landsize',
 'Lattitude',
 'Longtitude',
 'Propertycount']

In [58]:
my_cols = low_card_cols + numerical_cols

In [59]:
my_cols

['Type',
 'Method',
 'Regionname',
 'Rooms',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Landsize',
 'Lattitude',
 'Longtitude',
 'Propertycount']

In [62]:
X_train.head(1)

,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Regionname,Propertycount
12167,St Kilda,11/22 Charnwood Cr,1,u,S,hockingstuart,29/07/2017,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,Southern Metropolitan,13240.0


In [65]:
X_train_full = X_train.copy()
X_valid_full = X_valid.copy()
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [66]:
X_train_full.shape

(10864, 16)

In [67]:
X_train.shape

(10864, 12)

In [80]:
s = (X_train.dtypes == 'object')
s[s].index

Index(['Type', 'Method', 'Regionname'], dtype='object')

In [82]:
object_cols = list(s[s].index)
object_cols

['Type', 'Method', 'Regionname']

In [84]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


In [86]:
drop_X_train = X_train.select_dtypes(exclude=["object"])
drop_X_valid = X_valid.select_dtypes(exclude=["object"])
drop_X_train

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.98670,13240.0
6524,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.90050,6380.0
8413,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.82200,3755.0
2919,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.91580,8870.0
6043,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.82720,4217.0
...,...,...,...,...,...,...,...,...,...
13123,3,5.2,3056.0,3.0,1.0,212.0,-37.77695,144.95785,11918.0
3264,3,10.5,3081.0,3.0,1.0,748.0,-37.74160,145.04810,2947.0
9845,4,6.7,3058.0,4.0,2.0,441.0,-37.73572,144.97256,11204.0
10799,3,12.0,3073.0,3.0,1.0,606.0,-37.72057,145.02615,21650.0


In [87]:
score_dataset(drop_X_train,drop_X_valid,y_train,y_valid)

np.float64(175707.61156991488)

In [88]:
from sklearn.preprocessing import OrdinalEncoder
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()
my_ordinal = OrdinalEncoder()

label_X_train[object_cols] = my_ordinal.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = my_ordinal.transform(X_valid[object_cols])

In [89]:
score_dataset(label_X_train,label_X_valid,y_train,y_valid)

np.float64(165919.14549617787)

In [107]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
166111.84115541063


In [104]:
X_train.shape

(10864, 12)

In [ ]:
User_1  ==>  order creater (accept payment) ==> pending
user cancancel (Time)  => cancelled
    -> Refund   ---> Refunded

cater ---> reject   ===>  Rejected
            refund initiate
            super admin --> Refunded
             


cater -->  accept ---> procesing
cater -->  if accept  --->  deliverd
